# Batch anomaly detection with the Anomaly Detector API

# Batch anomaly detection with the Anomaly Detector API

The *Anomaly Detector* API enables you to monitor and detect abnormalities in your time series data with machine learning. The Anomaly Detector API adapts by automatically identifying and applying the best-fitting models to your data, regardless of industry, scenario, or data volume. Using your time series data, the API determines boundaries for anomaly detection, expected values, and which data points are anomalies.

Anomaly Detector provides two APIs that detect anomalies automatically in time series with simple parameters, which require no machine learning background. It is designed for the scenarios of operational monitoring, business KPI monitoring, and IoT monitoring. Using Anomaly Detector APIs, you can infuse anomaly detection capabilities into your own platform and business process.


## Use the Anomaly Detector Cognitive Service

Microsoft Azure includes a number of *cognitive services* that encapsulate common AI functions, including some that can help you build anomaly detection solutions.

The *Anomaly Detector* cognitive service provides an obvious starting point for having an out-of-the-box solution for detecting anomalies in Azure. It uses automatically selects the right algorithm for the varities of anomalies inculding seasonality, spikes, dips and trend changes.

### Create a Cognitive Services Resource

Let's start by creating a **Anomaly Detector** resource in your Azure subscription:

1. In another browser tab, open the Azure portal at https://portal.azure.com, signing in with your Microsoft account.
2. Click the **&#65291;Create a resource** button, search for *Anomaly Detector *, and create a **Cognitive Services** resource with the following settings:
    - **Name**: *Enter a unique name*.
    - **Subscription**: *Your Azure subscription*.
    - **Location**: *Choose any available region*:
    - **Pricing tier**: F0
    - **Resource group**: *Create a resource group with a unique name*.
    - Select the check box for *I confirm I have read and understood the notice below* as this service is still under preview. 
3. Wait for deployment to complete. Then go to your cognitive services resource, and on the **Overview** page, click the link to manage the keys for the service. You will need the endpoint and keys to connect to your cognitive services resource from client applications.

### Get the Key and Endpoint for your Cognitive Services resource

To use your cognitive services resource, client applications need its  endpoint and authentication key:

1. In the Azure portal, on the **Keys and Endpoint** page for your cognitive service resource, copy the **Key1** for your resource and paste it in the code below, replacing **YOUR_COG_KEY**.
2. Copy the **endpoint** for your resource and and paste it in the code below, replacing **YOUR_COG_ENDPOINT**.
3. Run the code in the cell below by clicking its green <span style="color:green">&#9655</span> button.

### Use this Jupyter notebook to start visualizing anomalies as a batch with the Anomaly Detector API in Python.

This notebook shows you how to send a batch anomaly detection request, and vizualize the anomalies found throughout the example data set. The graph created at the end of this notebook will display the following:
* Anomalies found throughout the data set, highlighted.
* The expected values versus the values contained in the data set.
* Anomaly detection boundaries 


In [ ]:
# To start sending requests to the Anomaly Detector API, paste your subscription key you received after creating Anomaly Detector resource. 
cog_key = 'YOUR_COG_KEY'

# Use the endpoint your received from overview section of the Anomaly Detector resource you created
# the endpoint is like https://westus2.api.cognitive.microsoft.com/, different by regions. 
# The code concats anomalydetector/v1.0/timeseries/entire/detect to the end point for the API calls
cog_endpoint = 'YOUR_COG_ENDPOINT'
cog_endpoint += 'anomalydetector/v1.0/timeseries/entire/detect'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

In [ ]:
# To start sending requests to the Anomaly Detector API, paste your subscription key you received after creating Anomaly Detector resource. 
subscription_key = 'fe41873c6f134d01a9ac223b1af02be7' 

# Use the endpoint your received from overview section of the Anomaly Detector resource you created
# the endpoint is like https://westus2.api.cognitive.microsoft.com/, different by regions, you need to concat anomalydetector/v1.0/timeseries/entire/detect

endpoint = 'https://anomaly-demo.cognitiveservices.azure.com/anomalydetector/v1.0/timeseries/entire/detect'

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')


# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
from bokeh.plotting import figure,output_notebook, show
from bokeh.palettes import Blues4
from bokeh.models import ColumnDataSource,Slider
import datetime
from bokeh.io import push_notebook
from dateutil import parser
from ipywidgets import interact, widgets, fixed
output_notebook()

In [ ]:
def detect(endpoint, subscription_key, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': cog_key}
    response = requests.post(cog_endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

In [ ]:
def build_figure(sample_data, sensitivity):
    sample_data['sensitivity'] = sensitivity
    result = detect(endpoint, subscription_key, sample_data)
    columns = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
          'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins'],
          'timestamp': [parser.parse(x['timestamp']) for x in sample_data['series']], 
          'value': [x['value'] for x in sample_data['series']]}
    response = pd.DataFrame(data=columns)
    values = response['value']
    label = response['timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Batch Anomaly Detection ({0} Sensitvity)".format(sensitivity), width=800, height=600)
    for anom in response['isAnomaly']:
        if anom == True and (values[index] > response.iloc[index]['expectedValues'] + response.iloc[index]['upperMargins'] or 
                         values[index] < response.iloc[index]['expectedValues'] - response.iloc[index]['lowerMargins']):
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        index = index+1
    upperband = response['expectedValues'] + response['upperMargins']
    lowerband = response['expectedValues'] -response['lowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend='Boundary')
    p.line(label, values, legend='Value', color="#2222aa", line_width=1)
    p.line(label, response['expectedValues'], legend='ExpectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    show(p, notebook_handle=True)

## Vizualizing anomalies throughout your data

The following cells call the Anomaly Detector API with two different example time series data sets, and different sensitivities for anomaly detection. Varying the sensitivity of the Anomaly Detector API can improve how well the response fits your data.

### Example 1: time series with an hourly sampling frequency


In [ ]:
# Hourly Sample
sample_data = json.load(open('sample_hourly.json'))
sample_data['granularity'] = 'hourly'
sample_data['period'] = 24
# 95 sensitivity
build_figure(sample_data,95)

In [ ]:
sample_data

In [ ]:
daily_data = json.load(open('csvjson.json'))
daily_data['granularity'] = 'minutely'

# 95 sensitivity
build_figure(daily_data,95)

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp']).dt.strftime('%Y-%m-%dT%H:%M%:%SZ')

In [ ]:
data.to_csv('art_daily_jumpsup2.csv')

In [ ]:
series = []
data_file = pd.read_csv('art_daily_jumpsup.csv', encoding='utf-8', parse_dates=[0])
for index, row in data_file.iterrows():
    series.append(Point(timestamp=row[0], value=row[1]))

In [ ]:
# 90 sensitivity
build_figure(sample_data,90)

In [ ]:
#85 sensitivity
build_figure(sample_data,85)

### Example 2: time series with an daily sampling frequency


In [ ]:
#daily sample
sample_data = json.load(open('sample.json'))
sample_data['granularity'] = 'daily'
# 95 sensitivity
build_figure(sample_data,95)

In [ ]:
# 90 sensitivity
build_figure(sample_data,90)

In [ ]:
# 85 sensitivity
build_figure(sample_data,80)